In [1]:
import pandas as pd
import geopandas as gpd 

# set working directory
import os
import sys
dname = '\\'.join(os.getcwd().split('\\')[:-1])
os.chdir(dname+'\\..')

In [2]:
df = pd.read_csv('src/Data/flextur_samlet_2024.csv', sep=';', decimal=',')
print(df.shape[0])
df.head(50)

gdf_fejl = gpd.read_file('src/Resultater/flextur_samlet_2024_vejnet.shp')
print(gdf_fejl.shape[0])
gdf_fejl.head(50)

83303
83303


,Rejser,Antal pass,Kommune1,Planet1,Kommune2,Planet2,Rejsetype,Periode,Retning,geometry
0,45,45,holstebro,hol208,struer,str207,adresserejse,2024 helÃ¥r,Summeret,"LINESTRING (484967.230 6248045.953, 484787.433..."
1,25,25,holstebro,hol208,struer,str207,adresserejse,2024 1. halvÃ¥r,Summeret,"LINESTRING (484967.230 6248045.953, 484787.433..."
2,26,26,struer,str207,holstebro,hol208,adresserejse,2024 helÃ¥r,Retningsbestemt,"LINESTRING (467921.900 6252522.164, 467922.762..."
3,14,14,struer,str207,holstebro,hol208,adresserejse,2024 1. halvÃ¥r,Retningsbestemt,"LINESTRING (467921.900 6252522.164, 467922.762..."
4,469,470,viborg,bje950,viborg,bje980,adresserejse,2024 helÃ¥r,Summeret,"LINESTRING (540617.732 6248875.198, 540618.811..."
5,207,208,viborg,bje950,viborg,bje980,adresserejse,2024 1. halvÃ¥r,Summeret,"LINESTRING (540617.732 6248875.198, 540618.811..."
6,210,211,viborg,bje980,viborg,bje950,adresserejse,2024 helÃ¥r,Retningsbestemt,"LINESTRING (541639.468 6251726.500, 541630.891..."
7,81,82,viborg,bje980,viborg,bje950,adresserejse,2024 1. halvÃ¥r,Retningsbestemt,"LINESTRING (541639.468 6251726.500, 541630.891..."
8,70,70,randers,pur011,randers,ran021,knudepunktsrejse,2024 helÃ¥r,Summeret,"LINESTRING (552952.832 6266835.981, 552935.193..."
9,30,30,randers,pur011,randers,ran021,knudepunktsrejse,2024 1. halvÃ¥r,Summeret,"LINESTRING (552952.832 6266835.981, 552935.193..."


In [3]:
# overfører geometry til oprindelig data da der kan være fejl i encoding for shapefil
gdf_new = gpd.GeoDataFrame(df, geometry=gdf_fejl.geometry)
gdf_new.drop(columns=['Fra X', 'Fra Y', 'Til X', 'Til Y'], inplace=True)
gdf_new.rename(columns={'Antal Rejser': 'rejser', 
                        'Antal passagere': 'passagere',
                        'Kommune1': 'kommune1',
                        'Planet1': 'planet1',
                        'Kommune2': 'kommune2',
                        'Planet2': 'planet2',
                        'Rejsetype': 'rejsetype',
                        'Periode': 'periode'}, 
               inplace=True)

# planetzoner med upper case
gdf_new['planet1'] = gdf_new['planet1'].str.upper()
gdf_new['planet2'] = gdf_new['planet2'].str.upper()

# kommuner og rejsetype med stort begyndelsesbogstav
gdf_new['kommune1'] = gdf_new['kommune1'].str.capitalize().replace({'Ringkøbing-skjern': 'Ringkøbing-Skjern',
                                                                    'Ikast-brande': 'Ikast-Brande'})
gdf_new['kommune2'] = gdf_new['kommune2'].str.capitalize().replace({'Ringkøbing-skjern': 'Ringkøbing-Skjern',
                                                                    'Ikast-brande': 'Ikast-Brande'})
gdf_new['rejsetype'] = gdf_new['rejsetype'].str.capitalize()

# anonymiser
gdf_new['rejser_u5'] = ''
gdf_new['pass_u5'] = ''
gdf_new.loc[gdf_new['rejser'] <= 5, 'rejser_u5'] = '5 eller derunder'
gdf_new.loc[gdf_new['passagere'] <= 5, 'pass_u5'] = '5 eller derunder'
gdf_new.loc[gdf_new['rejser'] <= 5, 'rejser'] = 0
gdf_new.loc[gdf_new['passagere'] <= 5, 'passagere'] = 0
gdf_new['rejser_txt'] = gdf_new[['rejser', 'rejser_u5']].apply(lambda row: row['rejser_u5'] if row['rejser']==0 else row['rejser'], axis=1)
gdf_new['pass_txt'] = gdf_new[['passagere', 'pass_u5']].apply(lambda row: row['pass_u5'] if row['passagere']==0 else row['passagere'], axis=1)
gdf_new.head(50)

,rejser,passagere,kommune1,planet1,kommune2,planet2,rejsetype,periode,Retning,geometry,rejser_u5,pass_u5,rejser_txt,pass_txt
0,45,45,Holstebro,HOL208,Struer,STR207,Adresserejse,2024 helår,Summeret,"LINESTRING (484967.230 6248045.953, 484787.433...",,,45,45
1,25,25,Holstebro,HOL208,Struer,STR207,Adresserejse,2024 1. halvår,Summeret,"LINESTRING (484967.230 6248045.953, 484787.433...",,,25,25
2,26,26,Struer,STR207,Holstebro,HOL208,Adresserejse,2024 helår,Retningsbestemt,"LINESTRING (467921.900 6252522.164, 467922.762...",,,26,26
3,14,14,Struer,STR207,Holstebro,HOL208,Adresserejse,2024 1. halvår,Retningsbestemt,"LINESTRING (467921.900 6252522.164, 467922.762...",,,14,14
4,469,470,Viborg,BJE950,Viborg,BJE980,Adresserejse,2024 helår,Summeret,"LINESTRING (540617.732 6248875.198, 540618.811...",,,469,470
5,207,208,Viborg,BJE950,Viborg,BJE980,Adresserejse,2024 1. halvår,Summeret,"LINESTRING (540617.732 6248875.198, 540618.811...",,,207,208
6,210,211,Viborg,BJE980,Viborg,BJE950,Adresserejse,2024 helår,Retningsbestemt,"LINESTRING (541639.468 6251726.500, 541630.891...",,,210,211
7,81,82,Viborg,BJE980,Viborg,BJE950,Adresserejse,2024 1. halvår,Retningsbestemt,"LINESTRING (541639.468 6251726.500, 541630.891...",,,81,82
8,70,70,Randers,PUR011,Randers,RAN021,Knudepunktsrejse,2024 helår,Summeret,"LINESTRING (552952.832 6266835.981, 552935.193...",,,70,70
9,30,30,Randers,PUR011,Randers,RAN021,Knudepunktsrejse,2024 1. halvår,Summeret,"LINESTRING (552952.832 6266835.981, 552935.193...",,,30,30


In [4]:
gdf_new.to_file('src/Resultater/flextur_samlet_2024_vejnet_anonymt.shp', driver='ESRI Shapefile', encoding='utf-8', src='EPSG:25832')